In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
products_df = spark.read \
    .format("parquet") \
        .load("abfss://bronze@salesstaradlsstorageacc.dfs.core.windows.net/products")

In [0]:
products_df.display()

In [0]:
products_df= products_df.withColumnRenamed("_rescued_data","Rescued_data")

In [0]:
null_df = products_df.filter(col("Rescued_data").isNotNull())

null_count= null_df.agg(count("Rescued_data").alias("county"))

In [0]:
null_count.display()


In [0]:
county =null_count.agg(max('county')).collect()[0][0]
print(county)

In [0]:
if null_count.agg(max('county')).collect()[0][0] ==0:
    products_df =products_df.drop("Rescued_data")
else:
    raise Exception("There are rescued records")



In [0]:
products_df.display()

In [0]:
products_df.createOrReplaceTempView("products")

##functions inside catalog

In [0]:
%sql


create or replace function
databricksete.bronze.discount_func(price double)
returns double
language sql
Return  price * 0.90



In [0]:
%sql
select product_id ,databricksete.bronze.discount_func(price) as discounted_price from products

In [0]:
discounted_df = products_df.withColumn("discounted_price",expr("databricksete.bronze.discount_func(price)"))

In [0]:
discounted_df.display()

In [0]:
%sql

create or replace function
databricksete.bronze.upper_func(p_brand string)
returns string
language python 
as
$$
    return p_brand.upper()

$$

In [0]:
products_df = products_df.withColumn("upper_products",expr("databricksete.bronze.upper_func(brand)"))

products_df.display()

In [0]:


products_df.write.mode("append").format("delta").save("abfss://silver@salesstaradlsstorageacc.dfs.core.windows.net/products")

In [0]:
regions_df = spark.read.format("parquet") \
    .load("abfss://bronze@salesstaradlsstorageacc.dfs.core.windows.net/region")

In [0]:
regions_df.display()

In [0]:
regions_df = regions_df.drop("_rescued_data")

In [0]:
regions_df.write.mode("append").format("delta").save("abfss://silver@salesstaradlsstorageacc.dfs.core.windows.net/regions")

In [0]:
dbutils.widgets.text("catalog_name","")

In [0]:
catalog_name = dbutils.widgets.get("catalog_name")

In [0]:
spark.sql(f""" 
create table if not exists {catalog_name}.silver.products_silver
using delta 
location "abfss://silver@salesstaradlsstorageacc.dfs.core.windows.net/products"
""" )